# Import All Necessary Modules And Setup Project

If you get any errors when importing these, ensure you run the commands:
```bash
$ python -m pip install -r requirements.txt
```
to install all necessary modules for this project. This command must be run from inside of this project directory.

It is recommended to use virtual environments for this project to ensure there is no conflicting package versions on your system.

Activate the virtual environment (if needed), run the pip install command, and then launch Jupyter Lab inside this project to get this project running.

In [ ]:
# Uncomment the following line to execute the pip install
# %pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np

# Visualization
from matplotlib import pyplot as plt
import seaborn as sns

from measure_incremental_development.compute import calculate_mid, classify_snapshots


## Load Datasets

In [ ]:
keystroke_df_unedited = pd.read_csv("data/keystrokes.csv")
student_df_unedited = pd.read_csv("data/students.csv")

#### Copy Datasets For Modification

This preserves the initial datasets, in case we ever need to bring an unedited column/row back into anything

In [ ]:
keystroke_df = keystroke_df_unedited.copy()
student_df = student_df_unedited.copy()

## Get DF Representing Single Student Submission And File

In [ ]:
def getFileInStudentSubmission(df, student, assignment, fileName):
    '''
    Returns a dataframe that shows all submission data for a single file on a given assignment, for a given student

    df - a dataframe of keystroke data
    student - a string representing the SubjectID to filter by
    assignment - a strinng representing the filename to filter by
    fileName - the name of the file to filter by

    Returns a COPY of the dataframe, containing only the selected student, assignment, and file
    '''
    f = df[
          (df.SubjectID == student)
        & (df.AssignmentID == assignment)
        & (df.CodeStateSection == fileName)
        ].copy()
    return f

In [ ]:
def getStudentSubmission(df, student, assignment):
    '''
    Returns a dataframe that shows all submission data for all files on a given assignment, for a given student

    df - a dataframe of keystroke data
    student - a string representing the SubjectID to filter by
    assignment - a string representing the filename to filter by
    
    Returns a COPY of the dataframe, containing only the selected student and assignment
    '''
    f = df[
          (df.SubjectID == student)
        & (df.AssignmentID == assignment)
        ].copy()

    return f

In [ ]:
def filterDownToRunAndEdits(df):
    return df[(
            (df.EventType == 'File.Edit')
          | (df.EventType == 'Run.Program')
          )]

In [ ]:
def filterDownToRunAndEditsAndPastes(df):
    return df[(
            (df.EventType == 'File.Edit')
          | (df.EventType == 'Run.Program')
          | (df.EventType == 'X-Paste')
          )]

In [ ]:
def getStudentSubmissionRunsAndEdits(df, student, assignment):
    '''
    Returns a dataframe that shows all submission data for a given assignment, for a given student, with only run and edit events

    df - a dataframe of keystroke data
    student - a string representing the SubjectID to filter by
    assignment - a strinng representing the filename to filter by
    '''
    return filterDownToRunAndEdits(getStudentSubmission(df, student, assignment))

In [ ]:
def getFileInStudentSubmissionRunsAndEdits(df, student, assignment, fileName):
    '''
    Returns a dataframe that shows all submission data for a single file on a given assignment, for a given student, with only run and edit events

    df - a dataframe of keystroke data
    student - a string representing the SubjectID to filter by
    assignment - a strinng representing the filename to filter by
    fileName - the name of the file to filter by
    '''

    return filterDownToRunAndEdits(getFileInStudentSubmission(df, student, assignment, fileName))

## Reconstruct Submissions

In [ ]:
def reconstructSingleFileDebugger(df):
    '''
    See a submission as it gets reconstructed, ignoring the Run events

    This is used primarily for debugging
    '''

    s = ''

    fileStateCount = 0
    for _,row in df[df.EventType=='File.Edit'].iterrows():
        i = int(row.SourceLocation)
        fileLengthRn = len(s)
        if i > fileLengthRn:
            print("DEBUG: Woah buddy, your cursor is out of bounds")


        insert = '' if pd.isna(row.InsertText) else row.InsertText
        delete = '' if pd.isna(row.DeleteText) else row.DeleteText
        s = s[:i] + insert + s[i+len(delete):]

        print(f"""\
================
StateNo : {fileStateCount}
{i=}
{insert=}
{delete=}
{row.EventID=}
================
{s}
""")
        fileStateCount += 1

    return s

In [ ]:
def reconstructFinalFile(df):
    '''
    Reconstruct a single file

    df - A dataframe containing a single student, for a single assignment, 
            and a single file of that assignment

    Returns: A string representing the final state of the file
    '''

    s = ''

    for _,row in df[df.EventType=='File.Edit'].iterrows():
        i = int(row.SourceLocation)

        insert = '' if pd.isna(row.InsertText) else row.InsertText
        delete = '' if pd.isna(row.DeleteText) else row.DeleteText
        s = s[:i] + insert + s[i+len(delete):]
    
    return s

In [ ]:
def reconstructFileAtRunEvents(df):
    '''
    Reconstruct a single file and save it's state at different run events

    df - A dataframe containing a single student, for a single assignment, 
            and a single file of that assignment. Must have edit events and run events

    Returns: A list containing strings representing the various states of the file at different run events
    '''

    fileStateAtRuns = []

    s = ''

    for _,row in df.iterrows():
        if row.EventType == 'File.Edit':
            i = int(row.SourceLocation)

            insert = '' if pd.isna(row.InsertText) else row.InsertText
            delete = '' if pd.isna(row.DeleteText) else row.DeleteText
            s = s[:i] + insert + s[i+len(delete):]
        elif row.EventType == 'Run.Program':
            fileStateAtRuns.append(s)

    fileStateAtRuns.append(s)

    return fileStateAtRuns

In [ ]:
def reconstructProjectAtRunEvents(df):
    '''
    Reconstruct a whole project and save it's state at different run events

    df - A dataframe containing a single student, for a single assignment, 
            and one+ files. Must have edit events and run events

    Returns: A dictionary containing keys for each file name, with values that are a list
                of strings, representing each file state at different run events

    NOTE: Some files may always be empty in their states. This happens when students make meaningful 
            edits to files, but delete it's contents before running their code
    '''

    # Remove NaN edit events to ensure that only meaningful edits are logged
    df = df[
          ( df.EventType != 'File.Edit' )
          | ( ( df.EventType == 'File.Edit' )
            & ~ ( ( pd.isna(df.InsertText) ) 
                & ( pd.isna(df.DeleteText) ) 
                )
              )
        ]

    allFileNames = df.CodeStateSection.unique()
    
    fileStates = dict()
    currentFileContents = dict()
    # Setup the fileStates dictionary for every file and the currentFileContents for each file
    for fileName in allFileNames:
        fileStates[fileName] = []
        currentFileContents[fileName] = ''

    for _,row in df.iterrows():
        if row.EventType == 'File.Edit':
            # Get the name of the file that was just edited
            thisFileName = row.CodeStateSection

            # Get edit information
            i = int(row.SourceLocation)
            insert = '' if pd.isna(row.InsertText) else row.InsertText
            delete = '' if pd.isna(row.DeleteText) else row.DeleteText

            # Make the edit and save the current state of the file
            s = currentFileContents[thisFileName]
            s = s[:i] + insert + s[i+len(delete):]
            currentFileContents[thisFileName] = s
            
        elif row.EventType == 'Run.Program':
            # Save all files at their current states

            # We only want to save when a run is started
            if row['X-Metadata'] != 'Start': continue

            for fileName in allFileNames:
                # Adding this extra newline causes the calculate_mid function to not crash
                #   Adding a single newline shouldn't affect the midscore significantly, but it does seem to have a notable affect
                fileStates[fileName].append(currentFileContents[fileName] + "\n")

    # Save all files at their final states
    for fileName in allFileNames:
        fileStates[fileName].append(currentFileContents[fileName])

    return fileStates

#### View Reconstructed Project

In [ ]:
def viewFinalReconstructedProject(reconstructedFiles):
    '''
    View the final file states that get reconstructed

    reconstructedFiles : A dictionary containing filenames as keys and a list
        of said file at various states

    Returns nothing, prints to the console
    '''

    for fileName in sorted(reconstructedFiles.keys()):
        print(40*"=")
        print(f"file = {fileName}")
        print(40*"=")
        print(reconstructedFiles[fileName][-1])

In [ ]:
def viewReconstructedProjectStates(reconstructedFiles):
    '''
    View the project at all the different run states that get reconstructed

    reconstructedFiles : A dictionary containing filenames as keys and a list
        of said file at various states

    Returns nothing, prints to the console
    '''
    numberOfStates = len(reconstructedFiles[reconstructedFiles.keys()[0]])

    for stateNo in range(numberOfStates):
        print("v" * 40)
        print(f"Project State : {stateNo if stateNo != numberOfStates - 1 else 'FINAL'}")
        print("^" * 40)
        for fileName in sorted(reconstructedFiles.keys()):
            print(40*"=")
            print(f"file = {fileName}")
            print(40*"=")
            print(reconstructedFiles[fileName][stateNo])

In [ ]:
student10df = getFileInStudentSubmissionRunsAndEdits(
    keystroke_df, 
    'Student10',
    'Assign10',
    'wordinator.py'
    )

In [ ]:
student10FileStates = reconstructFileAtRunEvents(student10df)

for i,fileState in enumerate(student10FileStates): 
    print("="*40)
    print(f"File State : {i}")
    print("="*40)
    print(fileState)

In [ ]:
student36df = getStudentSubmissionRunsAndEdits(
    keystroke_df, 
    'Student36',
    'Assign10',
    )

In [ ]:
display(student_df)

In [ ]:
student36FileStates = reconstructProjectAtRunEvents(student36df)
viewFinalReconstructedProject(student36FileStates)

In [ ]:
# Get all students and project list
def getStudentProjectList(students, df):
    eachProject = []
    runEvents = []
    allData = []
    for _,row in students.iterrows():
        for i in students.columns[2:10]:
            # print(row[1], i)
            # arr.append((row[1],i))
            temp = getStudentSubmission(df, row[1], i)
            # print(temp)
            eachProject.append(temp)
            test = reconstructProjectAtRunEvents(temp)
            runEvents.append(test)
            # print(test)
            allData.append((row[1], i, test))
    return eachProject, runEvents, allData
projects_df, run_events_df, final_data = getStudentProjectList(student_df, keystroke_df)

# getFileInStudentSubmissionRunsAndEdits
print(len(projects_df), len(run_events_df))

In [ ]:
# Get all students with a submission for the assignment
for student, assign, df in final_data:
    if len(df) > 0:
        print(student, assign)

In [ ]:
for student, assign, df in final_data:
    if len(df) > 0:
        # print(len(df))
        print(50*'=')
        print(student, assign)
        viewFinalReconstructedProject(df)
        print(50*'=')


## Add MID Library

- 0-2 Likely Incremental
- 2-2.5 Somewhat Incremental
- 2.5-3 Somewhat Non-Incremental
- 3+ Likely Non-Incremental

#### Calculate MID statistc for student and assignmemt

In [ ]:
SUBJECT_ID_KEY = 'SubjectID'
ASSIGNMENT_ID_KEY = 'AssignmentID'
MID_SCORE_KEY = 'MID_Score'
INCREMENTAL_KEY = 'Incremental' # boolean key in df, <=2.5 Incremental, >2.5 Non-Incremental
ASSIGNMENT_SCORE_KEY = 'AssignmentScore'
FINAL_SCORE_KEY = 'FinalScore'

In [ ]:
def remove_empty_at_start(file_states):
    '''
    Function to remove empty strings at beginning of list

    file_states: list of file states

    returns a clean list
    '''
    index = -1
    for i in range(len(file_states)):
        if file_states[i] != '':
            index = i
            break
    if index > -1:
        return file_states[index:]
    return file_states

In [ ]:
# Test for fun
fake_file_states = ['', '', '', '\nfake', '\nfake\n', '\nfak\n', '\nfake\n', '\n', '\n']
clean_fake_file_states = remove_empty_at_start(fake_file_states)
print(fake_file_states)
print(clean_fake_file_states)

calculate_mid(fake_file_states)

In [ ]:
def get_scores(student, assignment, student_df):
    '''
    Get the score the student got on the specified student

    student: a string representing the SubjectID
    assignment: a string representing the Assignment
    student_df: a df containing the students grades

    Returns: a dictionary of the student's score for the specified assignment, 
    and final score
    None if there is no 
    '''
    # TODO Maybe we want to add more scores, we can do that here
    scores = dict()
    if student in student_df[SUBJECT_ID_KEY].unique():
        row_loc = student_df.loc[student_df[SUBJECT_ID_KEY] == student]
        if assignment in row_loc:
            scores[ASSIGNMENT_SCORE_KEY] = row_loc[assignment].values[0]
            scores[FINAL_SCORE_KEY] = row_loc[FINAL_SCORE_KEY].values[0]
    return scores

In [ ]:
print(get_scores('Student10', 'Assign10', student_df))

In [ ]:
def get_mid_score_row(student, assignment, mid_score):
    '''
    Get a dictionary containing necessary information to build a row
    in the student dataframe with MID score
    
    student - a string representing the SubjectID to filter by
    assignment - a strinng representing the filename to filter by
    file - the name of the file in assignment
    
    Returns: A dictionary containing student, assignment, file, mid score
    and whether or not the returned score is incremental or not
    '''
    scores = get_scores(student, assignment, student_df)
    return {
        SUBJECT_ID_KEY: student,
        ASSIGNMENT_ID_KEY: assignment,
        MID_SCORE_KEY: mid_score,
        ASSIGNMENT_SCORE_KEY: scores[ASSIGNMENT_SCORE_KEY],
        FINAL_SCORE_KEY: scores[FINAL_SCORE_KEY],
        INCREMENTAL_KEY: 0 if mid_score > 2.5 else 1
    }


In [ ]:
def get_mid_score_all(final_data):
    '''
    Creates a dataframe for a student, assignment, mid score, final score, 
    assignment score and the incremental information

    final_data - multiple tuples containing a single student, for a single assignment, 
        and file states.

    Returns: A dataframe for all specified student containing the mid score
    and whether or not the student used incremental development
    '''
    mid_score_df = pd.DataFrame()
    for student, assignment, file_states in final_data:
        if len(file_states) == 0:
            continue
        all_file_states = []
        for _, state in file_states.items():
            clean_state = remove_empty_at_start(state)
            all_file_states += clean_state
        try:
            mid_score = calculate_mid(all_file_states)
            row = pd.DataFrame(get_mid_score_row(student, assignment, mid_score), index=[0])
            mid_score_df = pd.concat([mid_score_df, row], ignore_index=True)
        except Exception as e:
            print("Failed for student. Error:", e)
            print(len(all_file_states))
    return mid_score_df

In [ ]:
mid_df = get_mid_score_all(final_data)

In [ ]:
mid_df.to_csv('./data/mid_scores.csv')